# Calculate Sunset and Sunrise
Welcome to this ugly notebook! What you find here: Calculates the earliest and latest sunrise and sunset for:
- the current year
- if everything was in wintertime
- if everything was in summertime

Earliest sunrise is not during longest day. There may be a fancy formula to calculate this. We go bruth force. So we calculate the sunrise and sunset for each day in year 2025 and find the corresponding entry.

## ToDo
* Add Andorra
* Add Bosnia and Herzegovina
* Isle of man, Jersey, Guernsey
* Nie Nacht prüfen
* Hexagons

In [13]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from astral import Observer
from astral.sun import sun
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz
from tqdm import tqdm
import json
import consts

## Define

In [14]:
USE_GRID = 'hex'

## Import Data

In [15]:
if USE_GRID == 'nuts':
    gdf_grid = gpd.read_file(consts.PATH_MASTERNUTS)
    gdf_grid['geometry'] = gdf_grid['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))
    PATH_TIMEDATA = consts.PATH_TIMEDATA_NUTS
elif USE_GRID == 'hex':
    gdf_grid = gpd.read_file(consts.PATH_HEXAGON)
    PATH_TIMEDATA = consts.PATH_TIMEDATA_HEX
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Calculate

In [16]:
# Get all possible dates from 2025
dates = [datetime(2025, 1, 1) + timedelta(days=x) for x in range(365)]

timedata = []

def transform_date_to_winter_summer_time(sun_object, event):

    datetime_local = s[event].astimezone(local_tz)

    # Subtract half a year from local_date
    half_year_before = sun_object[event] - relativedelta(months=6)
    half_year_before = half_year_before.astimezone(local_tz)

    if bool(half_year_before.dst()) == False:
        # Wintertime, no DST
        datetime_winter = half_year_before.time()
        datetime_summer = datetime_local.time()
    else:
        # Summertime, DST
        datetime_winter = datetime_local.time()
        datetime_summer = half_year_before.time()  
       
    return {
            'utc': sun_object[event],
            'local': datetime_local,
            'time_winter': datetime_winter,
            'time_summer': datetime_summer,
        }

def find_earliest_and_latest_in_list(suns_per_day, event, timeperiod):
    return {
            'earliest': min(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'earliest_day': min(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date(),
            'latest': max(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'latest_day': max(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date()
        }

with tqdm(total=len(gdf_grid)) as pbar:
    for i, row in gdf_grid.iterrows():

        pbar.update()

        location = Observer(row.geometry.centroid.y, row.geometry.centroid.x)

        # Earliest sunrise is not the same as the longest day. We need to calculate
        # the earliest day. We could do that the proper way. Or we could just bruth
        # force it. You know what we are gonna do..?
        suns_per_day = {
            'sunrise': [],
            'sunset': []
        }

        local_tz = pytz.timezone(row.timezone)
        for d in dates:
            try:
                s = sun(location, date=d)
                suns_per_day['sunrise'].append(transform_date_to_winter_summer_time(s, 'sunrise'))
                suns_per_day['sunset'].append(transform_date_to_winter_summer_time(s, 'sunset'))
            except ValueError:
                pass

        # Now get the earliest sunrise for winter and summer time
        record = {
            'nuts_id': row['NUTS_ID'],
            'name': row['NAME_LATN'] if 'NAME_LATN' in row else 'hex',
            'sunrise': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'],
                    'latest': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local']
                }
            },
            'sunset': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'],
                    'latest': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local']
                }
            }
        }

        timedata.append(record)

# Store
json.dump(timedata, open(PATH_TIMEDATA, 'w', encoding='UTF-8'), ensure_ascii=False, indent=2, default=str)

print("🍩 Finito")

100%|██████████| 6001/6001 [01:45<00:00, 56.87it/s]


🍩 Finito
